---
<h1 align="center"><strong>Cálculo de dependencia cruzada para todos los países</strong></h1>
<h4 align="center"><strong>Manuel Alejandro Hidalgo y Jorge Díaz Lanchas</strong></h4>
<h4 align="center"><strong>Fundación Real Instituto Elcano</strong></h4>

---

In [2]:
import os
import gzip
import pandas as pd
import numpy as np
from numpy.linalg import inv
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path

***No ejecutar este código a menos que se quiera comprimir***

In [ ]:
def comprimir_dividir_archivo(archivo_original, tamano_maximo=100, directorio_salida=None):
    # Asegúrate de que el archivo original existe
    archivo_original = Path(archivo_original)
    if not archivo_original.exists():
        raise FileNotFoundError(f"No se encuentra el archivo: {archivo_original}")
    
    # Si no se especifica directorio de salida, usar src/data/raw/ITP/
    if directorio_salida is None:
        # Obtener el directorio raíz del proyecto (donde está src/)
        proyecto_root = Path(__file__).parent.parent.parent
        directorio_salida = proyecto_root / 'src' / 'data' / 'raw' / 'ITP'
    else:
        directorio_salida = Path(directorio_salida)
    
    # Crear el directorio de salida si no existe
    directorio_salida.mkdir(parents=True, exist_ok=True)
    
    # Abre el archivo original en modo de lectura binaria
    with open(archivo_original, 'rb') as f_in:
        # Lee el contenido del archivo original
        contenido = f_in.read()
        
        # Determina el número de partes necesarias
        num_partes = (len(contenido) + tamano_maximo - 1) // tamano_maximo
        
        # Divide el contenido en partes y escribe cada parte comprimida
        for i in range(num_partes):
            parte = contenido[i * tamano_maximo: (i + 1) * tamano_maximo]
            archivo_salida = directorio_salida / f'ITPD_E_R02.csv.parte{i}.gz'
            with gzip.open(archivo_salida, 'wb') as f_out:
                f_out.write(parte)
            print(f"Parte {i} creada en: {archivo_salida}")

# Tamaño máximo por parte (1GB)
tamano_maximo = 1000 * 1024 * 1024

try:
    # Ruta al archivo original
    archivo_original = Path(r'Datos/ITP/ITPD_E_R02.csv')
    
    # Comprimir y dividir el archivo original
    comprimir_dividir_archivo(archivo_original, tamano_maximo)
    print("Proceso completado con éxito")
except Exception as e:
    print(f"Error durante el proceso: {e}")

***Descomprimir, carga de datos y borrado de archivo***

La compresión se hace para poder trabajar con git sin porblemas de tamaño de ficheros.
Se descomprime, se importa y luego se borra el fichero descomprimido


In [3]:
"""
FASE 1: PREPARACIÓN Y CARGA DE DATOS
Este script procesa la base de datos International Trade and Production Database (ITP)
que viene dividida en múltiples archivos comprimidos
"""
def procesar_datos_itp():
    try:
        # Definición de rutas usando Path y la estructura de tu proyecto
        # Si estamos en un notebook, usamos una ruta relativa
        try:
            base_path = Path(__file__).parent.parent.parent
        except NameError:  # Estamos en un notebook
            base_path = Path.cwd().parent  # Asumiendo que el notebook está en /notebooks/

        source_directory = base_path / "src" / "data" / "raw" / "ITP"
        target_directory = base_path / "src" / "data" / "processed"
        target_filename = 'ITPD_E_R02.csv'

        # Imprimir las rutas para verificación
        print(f"Directorio fuente: {source_directory}")
        print(f"Directorio destino: {target_directory}")

        # Asegurar que los directorios existen
        target_directory.mkdir(parents=True, exist_ok=True)

        # Verificar que el directorio fuente existe
        if not source_directory.exists():
            raise FileNotFoundError(f"No se encuentra el directorio fuente: {source_directory}")

        # Listar archivos comprimidos
        chunk_filenames = sorted([f for f in os.listdir(source_directory) 
                         if f.startswith('ITPD_E_R02.csv.parte') and f.endswith('.gz')])

        # Control de errores: verificar que existen archivos para procesar
        if not chunk_filenames:
            raise FileNotFoundError(f"No se encontraron archivos .gz en {source_directory}")

        # Construir la ruta completa para el archivo combinado
        target_filepath = target_directory / target_filename

        print("Combinando archivos comprimidos...")
        with open(target_filepath, 'wb') as target_file:
            for chunk_filename in tqdm(chunk_filenames, desc="Procesando archivos"):
                chunk_filepath = source_directory / chunk_filename
                with gzip.open(chunk_filepath, 'rb') as chunk_file:
                    target_file.write(chunk_file.read())

        print("Leyendo archivo CSV...")
        # Usar chunks para manejar archivos grandes de manera eficiente
        itp = pd.read_csv(target_filepath, sep=",", chunksize=100000)
        
        # Procesar por chunks y filtrar año 2019
        chunks_2019 = []
        for chunk in tqdm(itp, desc="Filtrando datos de 2019"):
            chunk_2019 = chunk[chunk['year'] == 2019]
            chunks_2019.append(chunk_2019)
        
        # Combinar todos los chunks filtrados
        itp2019 = pd.concat(chunks_2019, ignore_index=True)

        # Limpieza: eliminar archivo temporal
        os.remove(target_filepath)
        print(f"Archivo temporal eliminado")

        # Obtener lista única de países importadores
        codigos_countries = list(itp2019['importer_iso3'].unique())
        print(f"Total de países únicos encontrados: {len(codigos_countries)}")

        return itp2019, codigos_countries

    except Exception as e:
        print(f"Error durante el procesamiento: {e}")
        raise

if __name__ == "__main__":
    try:
        data, countries = procesar_datos_itp()
        print("Procesamiento completado con éxito")
    except Exception as e:
        print(f"Error en la ejecución principal: {e}")

Directorio fuente: c:\Users\Usuario\Documents\Github\Seguridad económica\src\data\raw\ITP
Directorio destino: c:\Users\Usuario\Documents\Github\Seguridad económica\src\data\processed
Combinando archivos comprimidos...


Procesando archivos: 100%|██████████| 7/7 [00:54<00:00,  7.83s/it]


Leyendo archivo CSV...


Filtrando datos de 2019: 726it [04:36,  2.63it/s]


Archivo temporal eliminado
Total de países únicos encontrados: 237
Procesamiento completado con éxito


# Clase AnalisisDependenciaComercial - Documentación Detallada

## Descripción General
Esta clase implementa un análisis completo de dependencia comercial entre países, utilizando matrices de flujos comerciales para calcular índices de dependencia directa e indirecta.

## Estructura de la Clase

### Constructor
```python
def __init__(self, codigos_paises: List[str])
```
- **Propósito**: Inicializa la clase con la lista de países a analizar
- **Atributos**:
  - `self.codigos_paises`: Lista ordenada de códigos ISO3 de países
  - `self.matrices`: Diccionario para almacenar matrices de comercio por industria
  - `self.matrices_O`: Almacena las submatrices Omega para cálculos intermedios

### Método: crear_matriz_comercio
```python
def crear_matriz_comercio(self, grouped_data) -> Dict[str, pd.DataFrame]
```
- **Propósito**: Crea matrices de flujos comerciales para cada industria
- **Funcionamiento**:
  1. Valida que los datos tengan las columnas necesarias
  2. Para cada industria:
     - Crea una matriz base de ceros
     - Filtra transacciones válidas
     - Asigna valores de comercio a la matriz
- **Parámetros**:
  - `grouped_data`: DataFrame agrupado por industria
- **Retorna**: Diccionario de matrices de comercio por industria

### Método: mover_fila_columna
```python
@staticmethod
def mover_fila_columna(df: pd.DataFrame, nombre: str) -> pd.DataFrame
```
- **Propósito**: Reordena una matriz moviendo un país específico al final
- **Importancia**: Necesario para el cálculo de vectores de dependencia
- **Proceso**:
  1. Valida existencia del país en filas y columnas
  2. Crea nueva lista de índices con el país al final
  3. Reordena el DataFrame

### Método: calcular_vectores_ae
```python
def calcular_vectores_ae(self, pais: str) -> Tuple[Dict, Dict]
```
- **Propósito**: Calcula vectores de dependencia directa
- **Proceso**:
  1. Valida existencia de matrices y país
  2. Para cada industria:
     - Normaliza la matriz por columnas
     - Mueve el país analizado al final
     - Establece diagonal en ceros
     - Extrae vector de dependencia
- **Retorna**: Tupla con vectores de dependencia y matrices normalizadas

### Método: calcular_dependencia_total
```python
def calcular_dependencia_total(self, pais: str) -> pd.DataFrame
```
- **Propósito**: Calcula la dependencia total (directa + indirecta)
- **Proceso**:
  1. Obtiene vectores de dependencia directa
  2. Construye matrices Omega (submatrices)
  3. Calcula matrices inversas (I - Ω)^(-1)
  4. Multiplica vectores por matrices inversas
- **Matemática Subyacente**:
  - Usa la fórmula d = ae(I - Ω)^(-1)
  - Captura efectos directos e indirectos

### Método: get_summary_stats
```python
def get_summary_stats(self, dependencia: pd.DataFrame) -> pd.DataFrame
```
- **Propósito**: Genera estadísticas descriptivas de la dependencia
- **Estadísticas calculadas**:
  - Media
  - Mediana
  - Desviación estándar
  - Máximo
  - Mínimo

## Consideraciones Técnicas
1. **Manejo de Errores**:
   - Validación de datos de entrada
   - Control de divisiones por cero
   - Manejo de errores en inversión de matrices

2. **Eficiencia**:
   - Uso de NumPy para operaciones matriciales
   - Manejo eficiente de memoria con copy()
   - Vectorización donde es posible

3. **Precisión Numérica**:
   - Manejo de valores NaN
   - Normalización de matrices
   - Control de errores numéricos

## Ejemplo de Uso
```python
# Inicializar análisis
analisis = AnalisisDependenciaComercial(codigos_paises)

# Crear matrices de comercio
analisis.crear_matriz_comercio(grouped_data)

# Calcular dependencia para un país
dependencia = analisis.calcular_dependencia_total('ESP')

# Obtener estadísticas
stats = analisis.get_summary_stats(dependencia)
```

In [4]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from numpy.linalg import inv
from typing import Dict, List, Tuple
import warnings


class AnalisisDependenciaComercial:
    def __init__(self, codigos_paises: List[str]):
        self.codigos_paises = sorted(codigos_paises)  # Lista ordenada de códigos de países
        self.matrices = {}                            # Diccionario para almacenar matrices
        self.matrices_O = None                        # Para matrices O (presumiblemente matrices origen)
        self.resultados_dependencia = {}              # Para almacenar resultados del análisis
        
        
    def crear_matriz_comercio(self, grouped_data) -> Dict[str, pd.DataFrame]:
        """
        Crea matrices de comercio bilateral para cada industria a partir de datos agrupados.
        
        Esta función procesa datos de comercio internacional y crea una matriz para cada industria
        donde las filas representan países exportadores y las columnas países importadores.
        Los valores en la matriz representan el volumen de comercio entre cada par de países.
        
        Args:
            grouped_data: DataFrame agrupado por industria que contiene columnas:
                        - exporter_iso3: código ISO3 del país exportador
                        - importer_iso3: código ISO3 del país importador
                        - trade: valor del comercio bilateral
                        
        Returns:
            Dict[str, pd.DataFrame]: Diccionario donde:
                                    - keys: nombres de industrias
                                    - values: matrices de comercio bilateral
        
        Raises:
            ValueError: Si faltan columnas requeridas en los datos
        """
        # Inicializar diccionario vacío para almacenar las matrices de cada industria
        matrices = {}
        
        # Definir conjunto de columnas que deben estar presentes en los datos
        required_columns = {'exporter_iso3', 'importer_iso3', 'trade'}
        
        # Verificar si todas las columnas requeridas están presentes en los datos
        # grouped_data.obj accede al DataFrame original antes del groupby
        if not required_columns.issubset(grouped_data.obj.columns):
            raise ValueError(f"Los datos deben contener las columnas: {required_columns}")
        
        # Iterar sobre cada industria y su grupo de datos correspondiente
        # tqdm añade una barra de progreso para monitorear el avance
        for industry, group in tqdm(grouped_data, desc="Creando matrices de comercio"):
            # Crear una matriz vacía inicializada con ceros
            # Tanto filas como columnas son los códigos de países definidos en self.codigos_paises
            matrix_df = pd.DataFrame(
                0.0,  # Valor inicial para todas las celdas
                index=self.codigos_paises,    # Países exportadores en las filas
                columns=self.codigos_paises   # Países importadores en las columnas
            )
            
            # Filtrar solo las transacciones donde tanto exportador como importador
            # están en la lista de países definida (self.codigos_paises)
            valid_trades = group[
                group['exporter_iso3'].isin(self.codigos_paises) & 
                group['importer_iso3'].isin(self.codigos_paises)
            ]
            
            # Para cada transacción válida, asignar el valor de comercio
            # a la posición correspondiente en la matriz
            for _, row in valid_trades.iterrows():
                # .at es más eficiente que .loc para acceder a valores individuales
                matrix_df.at[row['exporter_iso3'], row['importer_iso3']] = row['trade']
            
            # Almacenar la matriz completada en el diccionario, usando el nombre
            # de la industria como clave
            matrices[industry] = matrix_df
        
        # Guardar las matrices en el atributo de la clase para uso posterior
        self.matrices = matrices
        
        # Devolver el diccionario de matrices
        return matrices
        
    @staticmethod
    def mover_fila_columna(df: pd.DataFrame, nombre: str) -> pd.DataFrame:
        """
        Mueve una fila y columna específica al final de la matriz.
        
        Args:
            df: DataFrame que queremos reordenar
            nombre: Nombre de la fila/columna que queremos mover al final
        """
        # Comprobar si el nombre existe en filas y columnas
        if nombre not in df.index or nombre not in df.columns:
            raise ValueError(f"'{nombre}' debe estar presente en filas y columnas")
        
        # Crear copia para no modificar el DataFrame original        
        df = df.copy()
        
        # Crear lista nueva de columnas: primero todas menos 'nombre', y 'nombre' al final
        cols = [col for col in df.columns if col != nombre] + [nombre]
        
        # Crear lista nueva de filas: primero todas menos 'nombre', y 'nombre' al final
        rows = [idx for idx in df.index if idx != nombre] + [nombre]
        
        # Reordenar el DataFrame con las nuevas listas de filas y columnas
        return df.reindex(columns=cols, index=rows)

    def calcular_vectores_ae(self, pais: str) -> Tuple[Dict, Dict]:
        if not self.matrices:
            raise ValueError("Debe llamar a crear_matriz_comercio primero")
        if pais not in self.codigos_paises:
            raise ValueError(f"País '{pais}' no encontrado en los códigos de países")
            
        vectores_ae = {}
        matrices_normalizadas = {}
        
        for industry, matrix in self.matrices.items():
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                
                column_sum = matrix.sum(axis=0)
                column_sum = column_sum.replace(0, np.nan)
                normalized_matrix = matrix.div(column_sum, axis=1)
                normalized_matrix = normalized_matrix.fillna(0)
                
                normalized_matrix = self.mover_fila_columna(normalized_matrix, pais)
                
                np.fill_diagonal(normalized_matrix.values, 0)
                
                matrices_normalizadas[industry] = normalized_matrix
                vectores_ae[industry] = normalized_matrix.iloc[-1][:-1]
        
        return vectores_ae, matrices_normalizadas

    def calcular_dependencia_total(self, pais: str) -> tuple[dict, pd.DataFrame]:
        """
        Calcula la dependencia total (directa + indirecta) de un país respecto a otros países
        para cada industria.

        Args:
            pais (str): Código del país para el que se calcula la dependencia

        Returns:
            tuple: (matrices_inversas, depend) donde:
                matrices_inversas (dict): Diccionario de matrices inversas de Leontief por industria
                depend (pd.DataFrame): Matriz de dependencias totales
        """
        # Obtener vectores de dependencia directa y matrices normalizadas
        vectores_ae, matrices_normalizadas = self.calcular_vectores_ae(pais)
        
        # Crear diccionario de matrices O (matrices sin el país analizado)
        matrices_O = {}
        for industry, matrix in matrices_normalizadas.items():
            if matrix is not None and not matrix.empty:
                if matrix.shape[0] > 1 and matrix.shape[1] > 1:
                    matrices_O[industry] = matrix.iloc[:-1, :-1]
        
        # Verificar si hay matrices válidas para procesar
        if not matrices_O:
            raise ValueError("No hay matrices válidas para procesar")
        
        # Guardar matrices_O como atributo de la clase
        self.matrices_O = matrices_O
        
        # Calcular las matrices inversas de Leontief
        matrices_inversas = {}
        for industry, matrix in matrices_O.items():
            matrix_values = matrix.fillna(0).values
            try:
                inverse = inv(np.eye(matrix_values.shape[0]) - matrix_values)
                matrices_inversas[industry] = inverse
            except np.linalg.LinAlgError:
                warnings.warn(f"No se pudo calcular la inversa para {industry}")
                continue
        
        # Calcular dependencia total
        dependencia = {}
        for industry, inverse_matrix in matrices_inversas.items():
            ae = vectores_ae[industry]
            resultado = np.dot(ae.fillna(0), inverse_matrix)
            dependencia[industry] = resultado
        
        # Crear DataFrame de dependencias
        depend = pd.DataFrame(dependencia).T
        depend.columns = next(iter(matrices_O.values())).columns
        
        # Asegurarse de devolver exactamente dos valores
        return matrices_O, matrices_inversas, depend

    def get_summary_stats(self, dependencia: pd.DataFrame) -> pd.DataFrame:
        return pd.DataFrame({
            'Media': dependencia.mean(),
            'Mediana': dependencia.median(),
            'Desv. Est.': dependencia.std(),
            'Máximo': dependencia.max(),
            'Mínimo': dependencia.min()
        })
    def calcular_dependencias_todos_paises(self, datos_comercio: pd.DataFrame) -> Dict[str, pd.DataFrame]:
        """
        Calcula las dependencias comerciales para todos los países en el conjunto de datos.
        
        Parámetros:
        -----------
        datos_comercio : DataFrame
            DataFrame con los datos de comercio bilateral (debe contener las columnas
            'exporter_iso3', 'importer_iso3', 'industry_descr', 'trade')
        
        Retorna:
        --------
        dict
            Diccionario donde las claves son los países y los valores son DataFrames
            con sus dependencias comerciales por industria
        """

        # Crear el directorio si no existe para guardar las matrices de dependencia
        output_dir = Path.cwd().parent / "src" / "data" / "processed" / "Matrices de Dependencia"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)


        # Validar columnas necesarias
        required_columns = {'exporter_iso3', 'importer_iso3', 'industry_descr', 'trade'}
        if not required_columns.issubset(datos_comercio.columns):
            raise ValueError(f"Los datos deben contener las columnas: {required_columns}")
        
        # Agrupar los datos por industria para el análisis
        grouped_data = datos_comercio.groupby('industry_descr')
        
        # Crear las matrices de comercio iniciales
        print("Creando matrices de comercio iniciales...")
        self.crear_matriz_comercio(grouped_data)
        
        # Procesar cada país
        print("Calculando dependencias para cada país...")
        for pais in tqdm(self.codigos_paises, desc="Procesando países"):
            try:
                # Calcular la dependencia total para el país actual
                dependencia = self.calcular_dependencia_total(pais)
                
                # Almacenar resultados para este país
                self.resultados_dependencia[pais] = dependencia

                # Guardar la matriz en un archivo
                output_path = os.path.join(output_dir, f"matriz_dependencia_de_{pais}.csv")
                dependencia[2].to_csv(output_path, sep=";") # dependencia[2] porque es una tupla y queremos el DataFrame
        
                
            except Exception as e:
                print(f"Error procesando país {pais}: {str(e)}")
        
        return self.resultados_dependencia
    
    def analizar_resultados(self) -> Dict[str, Dict[str, pd.DataFrame]]:
        """
        Analiza y resume los resultados de dependencia para todos los países.
        
        Retorna:
        --------
        dict
            Diccionario con diferentes análisis de los resultados, incluyendo:
            - Estadísticas por país
            - Rankings de dependencia
            - Análisis de concentración
        """
        if not self.resultados_dependencia:
            raise ValueError("No hay resultados para analizar. Ejecute calcular_dependencias_todos_paises primero.")
        
        analisis = {}
        
        # 1. Estadísticas básicas por país
        stats_por_pais = {}
        for pais, depend in self.resultados_dependencia.items():
            stats = pd.DataFrame({
                'Media': depend.mean(),
                'Mediana': depend.median(),
                'Max': depend.max(),
                'Min': depend.min(),
                'Std': depend.std()
            })
            stats_por_pais[pais] = stats
        analisis['estadisticas_por_pais'] = stats_por_pais
        
        # 2. Rankings de dependencia por industria
        rankings = {}
        for pais, depend in self.resultados_dependencia.items():
            # Ordenar industrias por nivel de dependencia media
            ranking = depend.mean(axis=1).sort_values(ascending=False)
            rankings[pais] = ranking
        analisis['rankings'] = rankings
        
        # 3. Análisis de concentración (Herfindahl)
        concentracion = {}
        for pais, depend in self.resultados_dependencia.items():
            # Calcular índice de Herfindahl por industria
            herfindahl = (depend ** 2).sum(axis=1)
            concentracion[pais] = herfindahl
        analisis['concentracion'] = concentracion
        
        return analisis
    
    def obtener_resumen_pais(self, pais: str) -> Dict[str, pd.DataFrame]:
        """
        Genera un resumen detallado para un país específico.
        
        Parámetros:
        -----------
        pais : str
            Código ISO3 del país a analizar
            
        Retorna:
        --------
        dict
            Diccionario con diferentes métricas y análisis para el país
        """
        if pais not in self.resultados_dependencia:
            raise ValueError(f"No hay datos disponibles para {pais}")
            
        resumen = {}
        
        # Obtener dependencia para el país
        depend = self.resultados_dependencia[pais]
        
        # 1. Top 10 industrias más dependientes
        top_10 = depend.mean(axis=1).sort_values(ascending=False).head(10)
        resumen['top_10_industrias'] = top_10
        
        # 2. Estadísticas generales
        stats = pd.DataFrame({
            'Media': depend.mean(),
            'Mediana': depend.median(),
            'Max': depend.max(),
            'Min': depend.min(),
            'Std': depend.std()
        })
        resumen['estadisticas'] = stats
        
        # 3. Concentración por industria
        concentracion = (depend ** 2).sum(axis=1)
        resumen['concentracion'] = concentracion
        
        return resumen
    

# Preparación de datos
result = data.copy()
codigos_paises = list(result['exporter_iso3'].unique())

# Crear instancia de la clase
analisis = AnalisisDependenciaComercial(codigos_paises)

In [5]:
grouped_data = result.groupby('industry_descr')
mat=analisis.crear_matriz_comercio(grouped_data)

Creando matrices de comercio: 100%|██████████| 170/170 [01:11<00:00,  2.37it/s]


In [8]:
df = mat['Mining of lignite']

In [10]:
df.to_csv(r'C:\Users\Usuario\Downloads\Matriz_Mining_of_lignite.csv', sep=";")

In [8]:
# Agrupar los datos y crear matrices de comercio
grouped_data = result.groupby('industry_descr')
analisis.crear_matriz_comercio(grouped_data)
# Calcular dependencias para todos los países
resultados_dependencia = analisis.calcular_dependencias_todos_paises(result)

Creando matrices de comercio: 100%|██████████| 170/170 [06:12<00:00,  2.19s/it]


Creando matrices de comercio iniciales...


Creando matrices de comercio: 100%|██████████| 170/170 [01:13<00:00,  2.32it/s]


Calculando dependencias para cada país...


Procesando países: 100%|██████████| 237/237 [17:13<00:00,  4.36s/it]  


In [5]:
# Agrupar los datos y crear matrices de comercio
grouped_data = result.groupby('industry_descr')
matrices_de_comercio = analisis.crear_matriz_comercio(grouped_data)

Creando matrices de comercio: 100%|██████████| 170/170 [03:20<00:00,  1.18s/it]


In [12]:
mat['Mining of lignite'].to_csv(r"C:\Users\Usuario\Downloads\matriz_comercio.csv", sep=";")
mat = mat['Mining of lignite']

In [36]:
vectores_ae, matrices_normalizadas = analisis.calcular_vectores_ae('IDN')

In [14]:
matrices_normalizadas['Mining of lignite'].to_csv(r"C:\Users\Usuario\Downloads\matriz_NORMALIZADA.csv", sep=";")

In [37]:
matrices_O, matrices_inversas_ARE, depend= analisis.calcular_dependencia_total('IDN')

In [16]:
matrices_O['Mining of lignite'].to_csv(r"C:\Users\Usuario\Downloads\matriz_O.csv", sep=";")

In [35]:
INV_ARE = matrices_inversas_ARE['Mining of lignite']

In [20]:

# Convert NumPy array to pandas DataFrame
df_lignite = pd.DataFrame(matrices_inversas['Mining of lignite'])

# Save to CSV
df_lignite.to_csv(r"C:\Users\Usuario\Downloads\matriz_inversa.csv", sep=";")

In [27]:
resultado = np.dot( matrices_inversas['Mining of lignite'], vectores_ae['Mining of lignite'].fillna(0))

In [22]:
vectores_ae['Mining of lignite'].to_csv(r"C:\Users\Usuario\Downloads\vector_ae.csv", sep=";")

In [20]:
depend.to_csv(r"C:\Users\Usuario\Downloads\matriz_dependencia.csv", sep=";")

## Crear un csv de datos para mostrar

In [14]:
import pandas as pd
import os

def transform_matrix(matrix_path):
    """
    Transforma una matriz de dependencia donde:
    - Primera columna: nombres de industrias
    - Resto de columnas: países (códigos ISO3)
    - Valores: dependencias
    """
    # Extraer el código del país proveedor del nombre del archivo
    supplier_country = matrix_path.split('matriz_dependencia_de_')[-1].split('.')[0]
    
    # Leer el CSV con la primera columna como índice
    df = pd.read_csv(matrix_path, sep=";",  index_col=0)
    
    # Convertir a formato largo
    df_long = df.reset_index().melt(
        id_vars=['index'],
        var_name='dependent_country',
        value_name='dependency_value'
    )
    
    # Renombrar la columna de industria
    df_long = df_long.rename(columns={'index': 'industry'})
    
    # Añadir país proveedor y redondear valores
    df_long['supplier_country'] = supplier_country
    df_long['dependency_value'] = df_long['dependency_value'].round(3)
    
    # Filtrar valores muy pequeños y donde dependent_country no sea supplier_country
    df_long = df_long[
        (df_long['dependency_value'] >= 0.005) & 
        (df_long['dependent_country'] != df_long['supplier_country'])
    ]
    
    # Reordenar columnas
    return df_long[['dependent_country', 'supplier_country', 'industry', 'dependency_value']]

def process_all_matrices(input_matrices: dict, clustering_file: Path):
    """
    Procesa las matrices directamente de memoria y añade información de clustering
    """
    all_dfs = []
    errors = []
    
    # Procesar matrices
    for pais, dependencia in input_matrices.items():
        try:
            # Obtener el DataFrame de la tupla (es el tercer elemento, índice 2)
            matriz = dependencia[2]  # Extraemos el DataFrame de la tupla
            
            # Transformar la matriz a formato largo
            df_long = matriz.reset_index().melt(
                id_vars=['index'],
                var_name='dependent_country',
                value_name='dependency_value'
            )
            
            # Resto del procesamiento igual
            df_long = df_long.rename(columns={'index': 'industry'})
            df_long['supplier_country'] = pais
            df_long['dependency_value'] = df_long['dependency_value'].round(3)
            
            df_long = df_long[
                (df_long['dependency_value'] >= 0.005) & 
                (df_long['dependent_country'] != df_long['supplier_country'])
            ]
            
            df_final = df_long[['dependent_country', 'supplier_country', 'industry', 'dependency_value']]
            all_dfs.append(df_final)
            print(f"Procesado: {pais}")
            
        except Exception as e:
            errors.append(f"Error en {pais}: {str(e)}")
    
    if errors:
        print("\nErrores encontrados:")
        for error in errors:
            print(error)
    
    if all_dfs:
        # Combinar todas las matrices
        combined_df = pd.concat(all_dfs, ignore_index=True)
        
        try:
            # Cargar datos de clustering
            clustering_data = pd.read_csv(clustering_file, sep=';')
            
            # Merge para país dependiente
            final_df = pd.merge(
                combined_df,
                clustering_data[['iso_d', 'cluster']],
                left_on='dependent_country',
                right_on='iso_d',
                how='left'
            )
            
            # Renombrar columna de cluster para país dependiente
            final_df = final_df.rename(columns={'cluster': 'comunidad_depend'})
            
            # Merge para país proveedor
            final_df = pd.merge(
                final_df,
                clustering_data[['iso_d', 'cluster']],
                left_on='supplier_country',
                right_on='iso_d',
                how='left'
            )
            
            # Renombrar columna de cluster para país proveedor
            final_df = final_df.rename(columns={'cluster': 'comunidad_supplier'})
            
            # Eliminar columnas redundantes
            final_df = final_df.drop(['iso_d_x', 'iso_d_y'], axis=1)
            
            # Guardar resultado final
            # En la parte donde guardamos el archivo final, cambiar:
            output_file = Path.cwd().parent / "src" / "data" / "processed" / "Dependencias consolidadas" / "dependencias_consolidadas.csv"
            output_file_gz = output_file.with_suffix('.csv.gz')
            output_file.parent.mkdir(parents=True, exist_ok=True)

            # Guardar comprimido
            final_df.to_csv(output_file_gz, index=False, compression='gzip')

            print(f"\nProceso completado. Guardado en: {output_file_gz}")
            print(f"Total registros: {len(final_df)}")
            return final_df
            
        except Exception as e:
            print(f"Error al procesar el archivo de clustering: {str(e)}")
            return combined_df
    else:
        raise ValueError("No se procesaron matrices correctamente")
# Uso:
clustering_file = Path.cwd().parent / "src" / "data" / "processed" / "comunidades" / "agglomerative_clustering_results.csv"

# Verificar que existe el archivo de clustering
if not clustering_file.exists():
    raise ValueError(f"El archivo de clustering {clustering_file} no existe")

# El diccionario resultados_dependencia ya contiene todas las matrices
combined = process_all_matrices(analisis.resultados_dependencia, clustering_file)

Procesado: ABW
Procesado: AFG
Procesado: AGO
Procesado: AIA
Procesado: ALB
Procesado: AND
Procesado: ARE
Procesado: ARG
Procesado: ARM
Procesado: ASM
Procesado: ATA
Procesado: ATF
Procesado: ATG
Procesado: AUS
Procesado: AUT
Procesado: AZE
Procesado: BDI
Procesado: BEL
Procesado: BEN
Procesado: BES
Procesado: BFA
Procesado: BGD
Procesado: BGR
Procesado: BHR
Procesado: BHS
Procesado: BIH
Procesado: BLM
Procesado: BLR
Procesado: BLZ
Procesado: BMU
Procesado: BOL
Procesado: BRA
Procesado: BRB
Procesado: BRN
Procesado: BTN
Procesado: BVT
Procesado: BWA
Procesado: CAF
Procesado: CAN
Procesado: CCK
Procesado: CHE
Procesado: CHL
Procesado: CHN
Procesado: CIV
Procesado: CMR
Procesado: COD
Procesado: COG
Procesado: COK
Procesado: COL
Procesado: COM
Procesado: CPV
Procesado: CRI
Procesado: CUB
Procesado: CUW
Procesado: CXR
Procesado: CYM
Procesado: CYP
Procesado: CZE
Procesado: DEU
Procesado: DJI
Procesado: DMA
Procesado: DNK
Procesado: DOM
Procesado: DZA
Procesado: ECU
Procesado: EGY
Procesado:

# DEPENDENCIA TOTAL

En este fragmento de código, se lleva a cabo un análisis de dependencia económica entre diferentes industrias utilizando datos de comercio internacional. 

En esta fase se calcula lo que sería la dependencia sin separar los países "terceros".

Inicialmente, se crea una lista de códigos de países que incluye "tercero". Luego, los datos se agrupan por descripción de la industria y se procesan para crear matrices de comercio internacional, las cuales representan las relaciones comerciales entre países en cada industria. Posteriormente, se normalizan estas matrices y se calcula un vector específico para cada industria. Además, se genera una matriz de dependencia económica utilizando la inversa de la matriz normalizada. Finalmente, se renombran las columnas de esta matriz con los códigos de países de la Unión Europea.


In [6]:
# Copiar la base de datos original
result = itp2019.copy()

# Crear una lista con todos los códigos de países
unique = result['exporter_iso3'].unique()
codigos_paises = list(unique)


# Primero, agrupamos los datos por 'industry_descr'
grouped_data = result.groupby('industry_descr')

#Obtengo matrices de flujos comerciales por industria y país

matrices = crear_matriz_comercio(grouped_data, codigos_paises)



# Diccionarios para almacenar resultados

resultados_dependencia = {}

# Iterar sobre la lista de países con una barra de progreso
for pais in tqdm(codigos_paises, desc="Procesando países"):
    # Obtener vectores ae y matrices normalizadas
    vectores_ae, matrices_normalizadas = calcular_vectores_ae(matrices, pais)
    
    # Construir matrices Omega
    matrices_O = obtener_submatrices(matrices_normalizadas)
    
    # Calcular las matrices inversas
    matrices_inversas = calcular_matrices_inversas(matrices_O)
    
    # Calcular dependencia
    depend = calcular_dependencia(vectores_ae, matrices_inversas)
    
    # Guardar los resultados en los diccionarios
    resultados_dependencia[pais] = depend

Creando matrices de comercio:  36%|███▌      | 61/170 [00:59<01:45,  1.03it/s]


KeyboardInterrupt: 

In [7]:

resultados_dependencia = {}

# Iterar sobre la lista de países con una barra de progreso
for pais in tqdm(codigos_paises, desc="Procesando países"):
    # Obtener vectores ae y matrices normalizadas
    vectores_ae, matrices_normalizadas = calcular_vectores_ae(matrices, pais)
    
    # Construir matrices Omega
    matrices_O = obtener_submatrices(matrices_normalizadas)
    
    # Calcular las matrices inversas
    matrices_inversas = calcular_matrices_inversas(matrices_O)
    
    # Calcular dependencia
    depend = calcular_dependencia(vectores_ae, matrices_inversas)
    
    # Guardar los resultados en los diccionarios
    resultados_dependencia[pais] = depend

Procesando países: 100%|██████████| 237/237 [27:58<00:00,  7.08s/it]  


## Construyo dataframes con estructura de panel para dependencias y pesos

Pesos

In [76]:

# Crear un diccionario para almacenar los DataFrames
dict_of_dfs = {}

# Agrupar por 'exporter_iso3'
grouped = result.groupby('exporter_iso3')

# Iterar sobre cada grupo
for exporter, group in grouped:
    # Pivotar el DataFrame
    pivot_df = group.pivot(index='industry_descr', columns='importer_iso3', values='trade')

    # Reemplazar NaN con ceros
    pivot_df = pivot_df.fillna(0)
    
    # Sumar las columnas
    col_sums = pivot_df.sum(axis=0)
    
    # Dividir cada celda por el total de su columna
    normalized_df = pivot_df.div(col_sums, axis=1)
    
    # Eliminar la columna que se llama igual que el valor de exporter_iso3
    if exporter in normalized_df.columns:
        normalized_df = normalized_df.drop(columns=[exporter])
    
    # Almacenar el DataFrame normalizado en el diccionario
    dict_of_dfs[exporter] = normalized_df
    
reshape_dfs = {}
for country, df in tqdm(dict_of_dfs.items(), desc="Procesando países"):
    # Resetear el índice y moverlo a una nueva columna llamada 'index_column'
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'industry_descr'}, inplace=True)
    # Reshape del DataFrame usando melt
    reshaped_df = df.melt(id_vars='industry_descr', var_name='importer', value_name='value')
    reshaped_df['exporter'] =country
    reshape_dfs[country] = reshaped_df

concatenated_w = pd.concat(reshape_dfs.values(), ignore_index=True).fillna(0)

Procesando países: 100%|██████████| 237/237 [00:02<00:00, 114.51it/s]


Dependencia

Creo un dataframe donde pongo columnas exportadores e importadores, industrias y pesos

In [77]:
   
reshape_dfs = {}
for country, df in tqdm(resultados_dependencia.items(), desc="Procesando países"):
    # Resetear el índice y moverlo a una nueva columna llamada 'index_column'
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'industry_descr'}, inplace=True)
    # Reshape del DataFrame usando melt
    reshaped_df = df.melt(id_vars='industry_descr', var_name='importer', value_name='dep')
    reshaped_df['exporter'] =country
    reshape_dfs[country] = reshaped_df

concatenated_dep = pd.concat(reshape_dfs.values(), ignore_index=True).fillna(0)


Procesando países: 100%|██████████| 237/237 [00:02<00:00, 91.28it/s] 


In [78]:
merged_df = pd.merge(concatenated_w, concatenated_dep, on=['industry_descr', 'importer', 'exporter'])

In [79]:
merged_df['dep_peso'] = merged_df['value']*merged_df['dep']

# Seleccionar las columnas relevantes
merged_df = merged_df[['importer', 'exporter', 'dep_peso']]

# Agrupar por "exporter" e "importer" y sumar los valores de "dep_peso"
merged_df_peso = merged_df.groupby(['exporter', 'importer']).sum().reset_index()


In [80]:
merged_df_peso.to_excel(r"C:\Users\Usuario\Downloads\import_dep.xlsx")